In [10]:
import matplotlib.pyplot as plt
import math
import numpy as np

# UNOBSERVABLE MODEL

### Waiting Time Function: 

In [11]:
def waiting_time(h1, h2, L1, L2, μ):
    W1 = (L1*h1 + L2*h2)/(μ*(μ - L1*h1))
    W2 = (L1*h1 + L2*h2)/((μ - L1*h1)*(μ - L1*h1 - L2*h2))
    return (W1, W2)

### Utility Function:

In [12]:
def calculate_utility(h1, h2, L1, L2, R1, c1, R2, c2, P, μ):
    U1 = R1 - P - c1*waiting_time(h1, h2, L1, L2, μ)[0]
    U2 = R2 - P - c2*waiting_time(h1, h2, L1, L2, μ)[1]
    return U1, U2

In [13]:
def check_utility(h1, h2, L1, L2, R1, c1, R2, c2, P, μ):
    U1, U2 = calculate_utility(h1, h2, L1, L2, R1, c1, R2, c2, P, μ)
    return (U1 >= 0 , U2 >= 0)

### Calculating Equilibrium Entering Probabilities: 

In [14]:
def calculate_h1_h2(L1, L2, R1, c1, R2, c2, P, μ):
    if R1 < P:
        sol = find_h1_h2(L1, L2, R1, c1, R2, c2, P, μ, h_1 = 0)
        return sol
    elif R2 < P:
        sol = find_h1_h2(L1, L2, R1, c1, R2, c2, P, μ, h_2 = 0)
        return sol
    else:
        if R1 == P or R2 == P:
            sol = find_h1_h2(L1, L2, R1, c1, R2, c2, P, μ)
            return sol  
        else:
            h1 = (μ-(c1/(R1-P))+(c2/(R2-P)))/L1
            h2 = (((-(R1-P)*c2*μ)/(c1*(R2-P))) + c1/(R1-P) - c2/(R2-P))/L2
            if 0 <= h1 <= 1 and 0 <= h2 <= 1:
                return (h1,h2)
            else:
                sol = find_h1_h2(L1, L2, R1, c1, R2, c2, P, μ)
                return sol

### Generated Algorithm to Find Equilibrium Entering Probabilities:

In [15]:
def find_h1_h2(L1, L2, R1, c1, R2, c2, P, μ, h_1 = None, h_2 = None):
    def is_dominated(point, other_points):
            x, y = point
            for other_x, other_y in other_points:
                if other_x > x and other_y >= y:
                    return True
            return False
    
    def find_key(dictionary, value):
        for key, val in dictionary.items():
            if val == value:
                return key
        return None

    def euclidean_distance(x1, y1, x2, y2):
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    if h_1 == None and h_2 == None:
        numbers = [(i/100, j/100) for i in range(101) for j in range(101)]
        numbers = [n for n in numbers if n[0] < μ/L1]
        numbers = [n for n in numbers if n[0]*L1 + n[1]*L2 < μ]
        utilities = {}
        for r in range(len(numbers)):
            utilities[numbers[r][0], numbers[r][1]] = check_utility(numbers[r][0], numbers[r][1], L1, L2, R1, c1, R2, c2, P, μ)
        filtered_dict = {k: v for k, v in utilities.items() if v == (True, True)}
        distinct_h1 = sorted({key[0] for key in filtered_dict.keys()})
        max_second_elements = {}
        for first_element in distinct_h1:
            max_second = None
            for key in filtered_dict.keys():
                if key[0] == first_element:
                    if max_second is None or key[1] > max_second:
                        max_second = key[1]
            max_second_elements[first_element] = max_second
        h1_values = list(max_second_elements.keys())
        h2_values = list(max_second_elements.values())
        zipped_values = list(zip(h1_values, h2_values))
        filtered_zipped = [point for point in zipped_values if not is_dominated(point, zipped_values)]
        if filtered_zipped == []:
                    solution_h = (0,0)
                    return(solution_h)
        else:
            remaining_utilities = {}
            for r in range(len(filtered_zipped)):
                remaining_utilities[filtered_zipped[r][0], filtered_zipped[r][1]] = calculate_utility(filtered_zipped[r][0], filtered_zipped[r][1], L1, L2, R1, c1, R2, c2, P, μ)
            list_U = remaining_utilities.values()
            min_distance = float('inf')
            closest_point = None
            for point in list_U:
                distance = euclidean_distance(point[0], point[1], 0, 0)
                if distance < min_distance:
                    min_distance = distance
                    closest_point = point
            solution_h = find_key(remaining_utilities, closest_point)
            return solution_h
    if h_1 == 0:
        list_h2 = [i/100 for i in range(101)]
        list_h2 = [h for h in list_h2 if h_1*L1 + h*L2 < μ]
        numbers = [(0,h2) for h2 in list_h2]
        utilities = {}
        for r in range(len(numbers)):
            utilities[numbers[r][0], numbers[r][1]] = check_utility(numbers[r][0], numbers[r][1], L1, L2, R1, c1, R2, c2, P, μ)[1]
        filtered_dict = {k: v for k, v in utilities.items() if v == (True)}
        max_h2 = None
        for key in filtered_dict.keys():
            if max_h2 is None or key[1] > max_h2:
                max_h2 = key[1]
        if max_h2 == None:
            solution_h = (0,0)
            return(solution_h)
        else:
            solution_h = 0, max_h2
            return solution_h
    
    if h_2 == 0:
        list_h1 = [i/100 for i in range(101)]
        list_h1 = [h for h in list_h1 if h < μ/L1]
        list_h1 = [h for h in list_h1 if h*L1 + h_2*L2 < μ]
        numbers = [(h1, 0) for h1 in list_h1]
        utilities = {}
        for r in range(len(numbers)):
            utilities[numbers[r][0], numbers[r][1]] = check_utility(numbers[r][0], numbers[r][1], L1, L2, R1, c1, R2, c2, P, μ)[0]
        filtered_dict = {k: v for k, v in utilities.items() if v == (True)}
        max_h1 = None
        for key in filtered_dict.keys():
            if max_h1 is None or key[0] > max_h1:
                max_h1 = key[0]
        if max_h1 == None:
            solution_h = (0,0)
            return(solution_h)
        else:
            solution_h = max_h1, 0
            return solution_h

### Calculating Total Revenue: 

In [16]:
def calculate_revenue(h1, h2, L1, L2, R1, c1, R2, c2, P, μ):
    L1_eq = h1*L1
    L2_eq = h2*L2
    revenue = (L1_eq + L2_eq) * P
    return revenue

### Revenue Maximization: 

In [17]:
def revenue_max_unobs(L1,L2,R1,c1,R2,c2,μ):
    P_values = np.linspace(0, max(R1,R2), 10*(max(R1,R2))+1)
    max_revenue = float('-inf')
    optimal_P = None
    revenue_list = {}
    h_data = {}
    for P in P_values:
        h1_h2 = calculate_h1_h2(L1, L2, R1, c1, R2, c2, P, μ)
        revenue = calculate_revenue(h1_h2[0], h1_h2[1], L1, L2, R1, c1, R2, c2, P, μ)
        revenue_list[P] = revenue
        h_data[P] = h1_h2
        if revenue is not None:
            if revenue > max_revenue:
                max_revenue = revenue
                optimal_P = P
    return max_revenue, optimal_P


### Utility Maximization: 

In [18]:
def utility_max_unobs(L1, L2, R1, c1, R2, c2, μ):
    P_values = np.linspace(0, max(R1,R2), 10*(max(R1,R2))+1)
    max_utility = float('-inf')
    optimal_P = None
    utility_list = {}
    data = {}
    for P in P_values:
        h1_h2 = calculate_h1_h2(L1, L2, R1, c1, R2, c2, P, μ)
        utility = L1*h1_h2[0]*calculate_utility(h1_h2[0], h1_h2[1], L1, L2, R1, c1, R2, c2, P, μ)[0] + L2*h1_h2[1]*calculate_utility(h1_h2[0], h1_h2[1], L1, L2, R1, c1, R2, c2, P, μ)[1]
        utility_list[P] = utility
        data[P] = h1_h2
        if utility is not None:
            if utility > max_utility:
                max_utility = utility
                optimal_P = P

    return max_utility, optimal_P


### Service Rate Optimization:

In [19]:
def service_rate_opt_unobs(μ_0, L1, L2, R1, c1, R2, c2):
    c = 1.5
    P_values = np.linspace(0, max(R1,R2), 10*(max(R1,R2))+1)
    μ_values = [1.5, 2.0,  2.5, 3.0,  3.5, 4.0,  4.5, 5.0] 
    max_revenue = float('-inf')
    optimal_param = None
    rate_list = {}
    for P in P_values:
        for μ in μ_values:
            h1_h2 = calculate_h1_h2(L1, L2, R1, c1, R2, c2, P, μ)
            revenue = calculate_revenue(h1_h2[0], h1_h2[1], L1, L2, R1, c1, R2, c2, P, μ) - c*max((μ-μ_0),0)
            rate_list[(P, μ)] = revenue
            if revenue is not None:
                if revenue > max_revenue:
                    max_revenue = revenue
                    optimal_param = (P, μ)
    return max_revenue, optimal_param


### Parameter Set Selection for the Experiments:

In [20]:
μ_values = [1.1 , 1.5, 2, 3.1]
R_values = [5, 10, 15]
c1_values = [3, 4, 5]
c2_values = [1, 2, 3]
parameters = [(L1, L2, R1, c1, R2, c2, μ) for R1 in R_values for c1 in c1_values for R2 in R_values for c2 in c2_values for L1 in [1] for L2 in [1]  for μ in μ_values]